In [1]:
import pandas as pd 
import numpy as np
import pytz 
import eemeter 
import glob
import re
import requests 
import json 
from os import getcwd, path 

# Packages to connect to Google Sheets 
import pickle 
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [2]:
class ProjectEvaluator: 
    def __init__(self, 
                 bill, 
                 update_temp_data=False, 
                 update_metadata=False):
        
        self.meter_data = pd.read_csv(bill)
        
        self.temperature_data = pd.read_csv("temperature_data.csv")
        self.temperature_data = self.temperature_data[["time", "temperature (F)"]]
        self.temperature_data["time"] = pd.to_datetime(self.temperature_data["time"])
        
        
        self.metadata = pd.read_csv("Metadata.csv")
        
        self.pod_id = re.findall('(?<=\/)(.*?)(?=\.)', bill)[0]
        self.bill_type = re.findall('.*(?=/)', bill)[0]
        
        temp = self.metadata[self.metadata["{}_PoD_ID".format(self.bill_type)] == self.pod_id]
        key =  temp["Project_Start_Date"].index.tolist()[0]
        
        self.project_start_date = pd.to_datetime(self.metadata["Project_Start_Date"][self.metadata["{}_PoD_ID".format(self.bill_type)] == self.pod_id][key])
        self.project_end_date = pd.to_datetime(self.metadata["Project_Completition_Date"][self.metadata["{}_PoD_ID".format(self.bill_type)] == self.pod_id][key])


    def prepare_data(self):
        self.meter_data = self.meter_data.iloc[::-1]
        self.meter_data["start"] = pd.to_datetime(self.meter_data["start"])
        meter_filter = (self.meter_data["start"] >= self.temperature_data.iloc[0]["time"])
        self.meter_data = self.meter_data.loc[meter_filter]
        self.meter_data = self.meter_data[["start", "value"]]
        
        
    def create_baseline_data(self): 
        '''
        This method consolidates all the necessary meter and temperature data 
        necessary to create the design matrix and baseline model to create the predictions. 
        the baseline meter data from the complete 
        SKIP if baseline meter data already exists. 
    
        '''        
        
        baseline_meter_filter = (self.meter_data["start"] <= self.project_start_date)
        self.baseline_meter_data = self.meter_data[baseline_meter_filter]        
        
        baseline_start = self.baseline_meter_data["start"].iloc[0].date()
        baseline_end = self.baseline_meter_data["start"].iloc[-1].date() + pd.offsets.Day(1)
        
        
        baseline_temp_mask = (self.temperature_data["time"] >= baseline_start) & (self.temperature_data["time"] <= baseline_end)
        self.baseline_temp_data = self.temperature_data.loc[baseline_temp_mask]
        self.baseline_temp_data = self.baseline_temp_data.rename(columns={"time": "dt",
                                                                          "temperature (F)": "tempF"})
        
        self.baseline_meter_data = self.baseline_meter_data.set_index("start")
        self.baseline_temp_data = self.baseline_temp_data.set_index("dt")
        
        baseline_meter_path = getcwd() + "/{}_".format(self.pod_id ) + "baseline_meter" 
        baseline_temp_path = getcwd() + "/{}_".format(self.pod_id ) + "baseline_temp" 
        
        self.baseline_meter_data = self.baseline_meter_data.to_csv(r'{}'.format(baseline_meter_path))
        self.baseline_temp_data = self.baseline_temp_data.to_csv(r'{}'.format(baseline_temp_path))
        
        self.ee_baseline_meter = eemeter.meter_data_from_csv(baseline_meter_path)
        
        self.ee_baseline_temp = eemeter.temperature_data_from_csv(baseline_temp_path) 
        self.ee_baseline_temp = self.ee_baseline_temp.asfreq('H')
        
                
        return self.ee_baseline_meter, self.ee_baseline_temp 

    
    def create_prediction_data(self): 
        '''
        '''
        prediction_meter_filter = (self.meter_data["start"] > self.project_end_date)
        self.prediction_meter_data = self.meter_data[prediction_meter_filter]
        
        self.prediction_start = self.project_end_date + pd.offsets.Day(1)
        self.prediction_end  = self.meter_data["start"].iloc[-1].date() + pd.offsets.Day(1)
        
        prediction_temp_filter = (self.temperature_data["time"] >= self.prediction_start) & (self.temperature_data["time"] <= self.prediction_end)
        self.prediction_temp_data = self.temperature_data.loc[prediction_temp_filter]
        self.prediction_temp_data = self.prediction_temp_data.rename(columns={"time": "dt",
                                                                              "temperature (F)": "tempF"})
        
        self.prediction_meter_data = self.prediction_meter_data.set_index("start")
        self.prediction_temp_data = self.prediction_temp_data.set_index("dt")
        
        prediction_meter_path = getcwd() + "/{}_".format(self.pod_id ) + "prediction_meter" 
        prediction_temp_path = getcwd() + "/{}_".format(self.pod_id ) + "prediction_temp" 
        
        self.prediction_meter_data = self.prediction_meter_data.to_csv(r'{}'.format(prediction_meter_path))
        self.prediction_temp_data = self.prediction_temp_data.to_csv(r'{}'.format(prediction_temp_path))
        
        self.ee_prediction_meter = eemeter.meter_data_from_csv(prediction_meter_path)
            
        self.ee_prediction_temp = eemeter.temperature_data_from_csv(prediction_temp_path) 
        self.ee_prediction_temp = self.ee_prediction_temp.asfreq('H')
        
        return self.ee_prediction_meter, self.ee_prediction_temp
    
    
    
    def estimate_metered_savings(self): 
        design_matrix = eemeter.create_caltrack_billing_design_matrix(self.ee_baseline_meter, 
                                                                      self.ee_baseline_temp)
        baseline_model = eemeter.fit_caltrack_usage_per_day_model(design_matrix)
        
        # Generating index for prediction 
        prediction_date_range = pd.date_range(start=self.prediction_start, 
                                              end=self.prediction_end, 
                                              freq='D')
        
        
        self.metered_savings, self.error_bands = eemeter.metered_savings(baseline_model, 
                                                              self.ee_prediction_meter,
                                                              self.ee_prediction_temp)
        
        return self.metered_savings 
        
    
    
    def establish_connection_to_spreadsheets(self, 
                                            spreadsheet_id, 
                                            range_name,
                                            connection_type, 
                                            body=None): 
        
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        
        SPREADSHEET_ID = spreadsheet_id 
        RANGE_NAME = range_name
        
        # This is where we make a batch get request 
        creds = None

        if path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        service = build('sheets', 'v4', credentials=creds)
        
        
        sheet = service.spreadsheets()
        
        if connection_type == "read": 
            result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                        range=RANGE_NAME).execute()
        if connection_type == "write":
            
            result = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, 
                                                        range=range_name,
                                                        valueInputOption="RAW", 
                                                        body=body).execute()
        print('{0} cells updated.'.format(result.get('updatedCells')))
        
        
        return result
        
    

    
    def update_project_metadata(self, 
                                spreadsheet_id, 
                                range_name): 
        '''
        Goes into the Google Sheets where the project metadata is stored 
        and pulls the new metadata. 
        '''
            
        result = self.establish_connection_to_spreadsheets(spreadsheet_id,
                                                            range_name, 
                                                           'read')
        
        raw_metadata = result.get('values', [])
        
        self.metadata = pd.DataFrame(raw_metadata[1:])
        self.metadata.columns = raw_metadata[0]
    
        
        return self.metadata
    


In [3]:
class ReportBuilder(ProjectEvaluator): 
    def __init__(self, files_path): 
        self.report = pd.DataFrame()
        self.savings_breakdown = dict()
        self.files_path = files_path 
        
    def build(self): 
        case_id = []
        reported_usage = [] 
        counterfactual_usage = [] 
        metered_savings = []

        
        for bill in glob.glob(self.files_path): 
            project_eval = ProjectEvaluator(bill)
            project_eval.prepare_data()
            project_eval.create_baseline_data()
            project_eval.create_prediction_data()
            project_eval.estimate_metered_savings()
            
            
            case_id.append(project_eval.pod_id)
            reported_usage.append(project_eval.metered_savings["reporting_observed"].sum())
            counterfactual_usage.append(project_eval.metered_savings["counterfactual_usage"].sum())
            metered_savings.append(project_eval.metered_savings["metered_savings"].sum())
            
            
            self.savings_breakdown[project_eval.pod_id] = project_eval.metered_savings
            

            
        self.report["Case_ID"] = case_id
        self.report["Reported_Usage"] = reported_usage
        self.report["Counterfactual_Usage"] = counterfactual_usage
        self.report["Metered_Savings"] = metered_savings
        
        
        return self.report, self.savings_breakdown
            
    def export_to_spreadsheets(self, 
                               spreadsheet_id,
                               range_name):
        

        # Read all existing data to ensure it doesn't get overwritten. 
        result = self.establish_connection_to_spreadsheets(spreadsheet_id,
                                                           range_name,
                                                           'read')
        raw_project_data = result.get('values', [])

    
        # Recreate data as a dataframe so it is readable/modifiable. 
        project_data = pd.DataFrame(columns=raw_project_data[0])
        data_by_cols = np.array(raw_project_data[1:]).T.tolist()

        for i in range(len(raw_project_data[1])):
            project_data[project_data.columns[i]] = data_by_cols[i]
        
        
        # Update the prev. dataframe with the values from the report.  
        project_data["Reported_Usage"] = self.report["Reported_Usage"]
        project_data["Counterfactual_Usage"] = self.report["Counterfactual_Usage"]
        project_data["Metered_Savings"] = self.report["Metered_Savings"]
        
        # Turn this updated df into the right format to export to sheets 
        values = []
        values.append(list(project_data.columns))

        for row in project_data.index:
            values.append(list(project_data.loc[row]))
            
        
        body = {
            'values': values 
            }
        
        # Export to sheets. 
        updated_df = self.establish_connection_to_spreadsheets(spreadsheet_id,
                                                               range_name, 
                                                               'write', 
                                                               body)                      
        return 

In [4]:
project_data_sheet_id = '16Vc8jTaFnDeaqqefoM62xu2zaUHvad9Fm27BueYieOw'
range_name = 'Main'

reports = ReportBuilder('Electricity/*?.csv')
reports.update_project_metadata(project_data_sheet_id, range_name)
summary, breakdown = reports.build()
reports.export_to_spreadsheets(project_data_sheet_id, range_name)


None cells updated.
creating design matrix for PoD IDN01000006827174
creating baseline model for PoD IDN01000006827174
creating design matrix for PoD IDN01000011073251
creating baseline model for PoD IDN01000011073251
creating design matrix for PoD IDN01000003628567
creating baseline model for PoD IDN01000003628567
None cells updated.
32 cells updated.
